In [20]:
import time
import os
import pandas as pd
import numpy as np
import sys
import multiprocessing

import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl
import fnmatch
import pickle
import datetime as dt
from datetime import timedelta
import glob
import fathon
from fathon import fathonUtils as fu
import itertools
import matplotlib as mpl
from scipy.stats import jarque_bera
from multiprocessing import Pool, freeze_support, Manager
import itertools
import freqopttest.kernel as kernel
from collections import defaultdict
colormap = plt.cm.RdBu
from scipy.cluster import hierarchy as hc

plt.style.use(os.path.join(mpl.get_configdir(), 'latexstyle3.mplstyle'))
'''
purpose of file is to load and change the results that we will use for the paper.
it does very little if any computation

'''

'\npurpose of file is to load and change the results that we will use for the paper.\nit does very little if any computation\n\n'

In [2]:

# Locations

t7 = '/media/ak/T71/August11th2022Experiments'  # little red t7 drive
folders = os.listdir(t7)  # Experiments folders
returns = os.path.join(t7, 'Returns')  # Returns folder
expOne = os.path.join(t7, 'ExperimentOne')  # Experiment One Folders
expTwo = os.path.join(t7, 'ExperimentTwo')
mfdfa_dfs = os.path.join(t7, 'mfdfaDataFrames')
ExperimentInputFiles = os.path.join(t7, 'ExperimentInputFiles')
QuadMMDOutputFilesLoc = os.path.join(expOne, 'QuadMMDOutputFiles')  # Quad MMD Results
LinearMMDOutputFilesLoc = os.path.join(expOne, 'TestMMDOutputFiles')  # Linear MMD Results
# symbolChoice = 'RX1'
# rx1 = os.path.join(ExperimentInputFiles, str(symbolChoice))
figures = '/home/ak/Documents/Papers/StylisedFactsPaper/figures'

LinearMMDOutputFiles = os.listdir(LinearMMDOutputFilesLoc)

In [3]:
folderCh = figures
os.path.isdir(folderCh)

True

In [4]:
symbolChoice = 'FB1'
barChoice ='calendar'
symbolBarChoice = str(symbolChoice) + "_" + str(barChoice)
SymbolLinearMMDOutputFiles = [f for f in LinearMMDOutputFiles if str(symbolBarChoice) in f]
variable_choice = 'tau'
variableChoiceFiles = [f for f in [f for f in SymbolLinearMMDOutputFiles if str(variable_choice) in f] if
                       str('_linear_') in f]

In [7]:

variableChoiceFileIdx = 0
variableChFileLoc = os.path.join(LinearMMDOutputFilesLoc, variableChoiceFiles[variableChoiceFileIdx])

FileDict = pd.read_pickle(variableChFileLoc)
shifts = list(FileDict.keys())

In [37]:
widths_dict

{0: [0.2759762398092724,
  0.318849892936312,
  0.3683840837014036,
  0.4256135445892378,
  0.4917337565665433,
  0.5681259217922896,
  0.6563858159057678,
  0.7583571225954363,
  0.8761699467829343,
  1.0122853109340988,
  1.1695465639918987,
  1.3512387768257408,
  1.5611573649240107,
  1.8036873718069673,
  2.083895069331642,
  2.4076337883511445,
  2.781666190452302,
  3.2138055349374897,
  3.7130788920130655,
  4.289915711586808,
  4.956365686736555,
  5.7263504628562725,
  6.615954450496787,
  7.643760816765577,
  8.831239673896716],
 1: [0.30385020145126956,
  0.35105414969190774,
  0.4055913585947482,
  0.4686010699805315,
  0.5413995099592398,
  0.6255073839167348,
  0.7226816429217198,
  0.834952185129694,
  0.9646642588489893,
  1.1145274530374722,
  1.287672298605029,
  1.4877156628810395,
  1.7188363032887302,
  1.9858621584866025,
  2.2943712004240897,
  2.650807954036046,
  3.062618118586018,
  3.5384041027982436,
  4.088104722791865,
  4.723202816573858,
  5.45696511200

In [13]:
widths_dict = {idxShift: FileDict[shifts[idxShift]]['widths'] for idxShift in range(0, len(shifts))}
powers_dict = {idxShift: FileDict[shifts[idxShift]]['powers'] for idxShift in range(0, len(shifts))}
med_dict = {idxShift: FileDict[shifts[idxShift]]['med'] for idxShift in range(0, len(shifts))}
besti_dict = {idxShift: FileDict[shifts[idxShift]]['besti'] for idxShift in range(0, len(shifts))}

test_variance_dict = {idxShift: FileDict[shifts[idxShift]]['test_variance'] for idxShift in range(0, len(shifts))}

compute_lin_unb_est_dict = {idxShift: FileDict[shifts[idxShift]]['compute_unbiased_linear_estimator'] for idxShift
                            in range(0, len(shifts))}
h0_rejected = {idxShift: FileDict[shifts[idxShift]]['test_result']['h0_rejected'] for idxShift in
               range(0, len(shifts))}
p_stat = {idxShift: FileDict[shifts[idxShift]]['test_result']['test_stat'] for idxShift in range(0, len(shifts))}
moment_one = {idxShift: FileDict[shifts[idxShift]]['two_moments'][0] for idxShift in range(0, len(shifts))}
moment_two = {idxShift: FileDict[shifts[idxShift]]['two_moments'][1] for idxShift in range(0, len(shifts))}


In [44]:
results_table = dict()
perce = list(h0_rejected.values()).count(True) / len(list(h0_rejected.values()))

print(widths_dict.keys())
kernels_Dict = dict()


dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])


In [45]:


for keyIdx in list(widths_dict.keys()):
    kernels_Dict[keyIdx] = ([kernel.KGauss(w ** 2) for w in widths_dict[keyIdx]][besti_dict[keyIdx]].sigma2)

    results_table['Unbiased Linear Estimator(Mean)'] = round(np.mean(list(compute_lin_unb_est_dict.values())), 3)
    results_table['Unbiased Linear Estimator(StD)'] = round(np.std(list(compute_lin_unb_est_dict.values())), 3)
    results_table['P-Stat(Mean)'] = round(np.mean(list(p_stat.values())), 3)
    results_table['P-Sta[keyIdx]t(StD)'] = round(np.std(list(p_stat.values())), 3)
    results_table['Test Variance(Mean)'] = round(np.mean(list(test_variance_dict.values())), 3)
    results_table['Test Variance(StD)'] = round(np.std(list(test_variance_dict.values())), 3)
    results_table['Best Kernel(\\sigma^2)'] = np.mean(list(kernels_Dict.values()))
    results_table['Test Variance'] = round(np.mean(np.abs(list(test_variance_dict.values()))), 3)
    results_table['Moment One'] = round(np.mean(list(moment_one.values())), 3)
    results_table['Moment Two'] = round(np.mean(list(moment_two.values())), 3)
    results_table['H0 Rejected(%)'] = "{0:.0%}".format(perce)

    print(results_table)
        save_loc = os.path.join(LinearMMDOutputFilesLoc, str(symbolChoice)+'_'+str(barChoice)+'_'+str(variable_choice)+'_Results_Table_LinearMMD.pkl')
        pickle.dump(results_table, open(save_loc, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)


{'Unbiased Linear Estimator(Mean)': 0.196, 'Unbiased Linear Estimator(StD)': 0.357, 'P-Stat(Mean)': 0.196, 'P-Sta[keyIdx]t(StD)': 0.357, 'Test Variance(Mean)': -0.156, 'Test Variance(StD)': 0.468, 'Best Kernel(\\sigma^2)': 0.7676737756456098, 'Test Variance': 0.16, 'Moment One': 0.196, 'Moment Two': 0.275, 'H0 Rejected(%)': '100%'}
{'Unbiased Linear Estimator(Mean)': 0.196, 'Unbiased Linear Estimator(StD)': 0.357, 'P-Stat(Mean)': 0.196, 'P-Sta[keyIdx]t(StD)': 0.357, 'Test Variance(Mean)': -0.156, 'Test Variance(StD)': 0.468, 'Best Kernel(\\sigma^2)': 0.42999936028379343, 'Test Variance': 0.16, 'Moment One': 0.196, 'Moment Two': 0.275, 'H0 Rejected(%)': '100%'}
{'Unbiased Linear Estimator(Mean)': 0.196, 'Unbiased Linear Estimator(StD)': 0.357, 'P-Stat(Mean)': 0.196, 'P-Sta[keyIdx]t(StD)': 0.357, 'Test Variance(Mean)': -0.156, 'Test Variance(StD)': 0.468, 'Best Kernel(\\sigma^2)': 0.2929853024754318, 'Test Variance': 0.16, 'Moment One': 0.196, 'Moment Two': 0.275, 'H0 Rejected(%)': '100%

In [46]:
print(results_table)



{'Unbiased Linear Estimator(Mean)': 0.196, 'Unbiased Linear Estimator(StD)': 0.357, 'P-Stat(Mean)': 0.196, 'P-Sta[keyIdx]t(StD)': 0.357, 'Test Variance(Mean)': -0.156, 'Test Variance(StD)': 0.468, 'Best Kernel(\\sigma^2)': 0.15686522850336254, 'Test Variance': 0.16, 'Moment One': 0.196, 'Moment Two': 0.275, 'H0 Rejected(%)': '100%'}


In [47]:
kernels_Dict

{0: 0.7676737756456098,
 1: 0.0923249449219771,
 2: 0.018957186858708616,
 3: 0.011739090117031729,
 4: 0.014674490727934572,
 5: 0.08989196242236407,
 6: 0.8721691306132449,
 7: 0.007601783537746004,
 8: 0.25363543215638573,
 9: 0.008313521197239188,
 10: 0.010273518311065993,
 11: 0.007670869568662879,
 12: 0.007670869568662879,
 13: 0.02241082852665218,
 14: 0.017202056751183466,
 15: 0.02375813308893157,
 16: 0.2905914535221546,
 17: 0.32200652929049184,
 18: 0.014204873223410035,
 19: 0.01896123090204047,
 20: 0.4224381176191156}

In [39]:
compute_lin_unb_est_dict

{0: 0.017806436602234695,
 1: 0.4799473592753356,
 2: 1.3596416511500062,
 3: 0.925180235659142,
 4: 0.0838349282765875,
 5: 0.0017176132916589973,
 6: 0.5771949966404296,
 7: 0.44532930321416614,
 8: 0.0011881306922626106,
 9: 0.006274134417153626,
 10: 0.007088269423329277,
 11: 0.008369962785079614,
 12: 0.0191547618414745,
 13: 0.0036916353553320155,
 14: 0.009283349484789299,
 15: 0.11265572792356769,
 16: 0.010899128767965127,
 17: 8.693911200129253e-05,
 18: 0.002260263219599242,
 19: 0.044022400562255294,
 20: 0.005434409213968837}